/config.yaml

model_evaluation:
  root_dir: artifacts/model_evaluation
  metrics_file_name: metrics.json
  model_path: artifacts/model_training/lstm_model.h5
  X_test_path: artifacts/data_transformation/X_test.npy
  y_test_path: artifacts/data_transformation/y_test.npy

In [ ]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    metrics_file_name: str
    model_path: str
    X_test_path: str
    y_test_path: str

In [ ]:
from finance_ml.constants import *
from finance_ml.utils.common import read_yaml, create_directories
from finance_ml.entity.config_entity import DataIngestionConfig, DataTransformationConfig, ModelTrainingConfig, ModelTrainingParams, ModelEvaluationConfig 
import os

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
    config = self.config.model_evaluation

    create_directories([config.root_dir])
    
    evaluation_config = ModelEvaluationConfig(
        root_dir=config.root_dir,
        metrics_file_name=config.metrics_file_name,
        model_path=config.model_path,
        X_test_path=config.X_test_path,
        y_test_path=config.y_test_path
    )

    return evaluation_config

In [ ]:
import os
import numpy as np
from tensorflow import keras
import mlflow
from pathlib import Path

from finance_ml.entity.config_entity import ModelEvaluationConfig
from finance_ml.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate(self):
        # Use config paths for model and data
        model_path = getattr(self.config, "model_path", os.path.join("artifacts", "model_training", "lstm_model.h5"))
        X_test_path = getattr(self.config, "X_test_path", os.path.join("artifacts", "data_transformation", "X_test.npy"))
        y_test_path = getattr(self.config, "y_test_path", os.path.join("artifacts", "data_transformation", "y_test.npy"))

        if not (os.path.exists(model_path) and os.path.exists(X_test_path) and os.path.exists(y_test_path)):
            raise FileNotFoundError("Model or test data files not found. Please check the paths.")

        model = keras.models.load_model(model_path)
        X_test = np.load(X_test_path)
        y_test = np.load(y_test_path)

        # Evaluate the model
        results = model.evaluate(X_test, y_test, verbose=0)
        if isinstance(results, list) and len(results) >= 2:
            loss, rmse = results[:2]
        else:
            raise ValueError("Model evaluation did not return expected metrics (loss, rmse).")

        # Log metrics to MLflow
        with mlflow.start_run():
            mlflow.log_metric("test_loss_mae", float(loss))
            mlflow.log_metric("test_rmse", float(rmse))
            # Optionally log the model
            mlflow.keras.log_model(model, "model")

        # Save metrics to a JSON file
        metrics = {
            "test_loss_mae": float(loss),
            "test_rmse": float(rmse)
        }
        metrics_file_path = Path(self.config.root_dir) / self.config.metrics_file_name
        save_json(metrics_file_path, metrics)

In [ ]:
# 
from finance_ml.config.configuration import configurationManager
from finance_ml.component.model_evaluation import ModelEvaluation
from finance_ml import logger

STAGE_NAME= "Model Evaluation stage"

class ModelEvaluationPipeline:
    def __init__(self):
        pass

    def main(self):
        try:
            config_manager = ConfigurationManager()
            evaluation_config = config_manager.get_model_evaluation_config()
            
            model_evaluation = ModelEvaluation(config = evaluation_config)
            model_evaluation.evaluate()

            logger.info (f"{STAGE_NAME} Completed successfully")

        except Exception as e:
            logger.exception(e)
            raise e
        
if __name__ == '__main__':
    try:
        logger.info(f'>>>>>>>>> stage {STAGE_NAME} started <<<<<<<<<')
        obj = ModelEvaluationPipeline()
        obj.main()
        logger.info(f'>>>>>>>>> stage {STAGE_NAME} Completed <<<<<<<<<\n\nx==========x')
    except Exception as e:
        logger.exception(e)
        raise e